In [125]:
#Read in data from a file as pandas dataframe
import sys
import pandas as pd

In [ ]:
data = pd.read_csv("WA_Crash_Summary.csv")
data.head()

In [ ]:
data.describe()

In [128]:
#Remove spaces at end of column Secondary Trafficway
data['Secondary Trafficway'] = data['Secondary Trafficway'].str.strip()

#Show unique values for Secondary Trafficway
data['Secondary Trafficway'].unique()

#Remove columns expect for the following columns 'Report Number', 'Crash Date/Time', 'City', 'Primary Trafficay', 'Mile Post', 'City'
data = data[['Collision Report Number', 'Collision Date', 'City', 'Primary Trafficway', 'Secondary Trafficway', 'Mile Post']]

#Sort by City and then by Mile Post
data = data.sort_values(by=['City', 'Mile Post'])

## Mile Post Filter for I-5 Bellingham

Milepost 241 on I-5 in Washington state is located near Bellingham, just south of the Lake Samish area. It is situated between the Bow Hill Road and Nulle Road exits. This is a more rural area with easy access to outdoor spots like Lake Samish and Chuckanut Mountain.

The last milepost on I-5 before the Canadian border is Milepost 276. This is just before the Peace Arch Border Crossing, where I-5 ends and connects to British Columbia's Highway 99 in Canada.

Here is a list of the mileposts between 241 and 276 on I-5 in Washington State:

- Milepost 241: Near Lake Samish, south of Bellingham
- Milepost 242: Nulle Road exit
- Milepost 243: Chuckanut Drive exit (SR 11)
- Milepost 250: Near Bellingham (South Bellingham area)
- Milepost 252: Samish Way / Fairhaven Parkway exit
- Milepost 253: Lakeway Drive exit
- Milepost 254: Iowa Street / James Street exit
- Milepost 255: Sunset Drive exit (SR 542, Mt. Baker Highway)
- Milepost 256: Meridian Street exit (SR 539)
- Milepost 260: Slater Road exit
- Milepost 262: Near Ferndale (Main Street exit)
- Milepost 266: Grandview Road exit
- Milepost 270: Near Blaine, Birch Bay-Lynden Road exit
- Milepost 274: Near Blaine, D Street exit
- Milepost 276: Peace Arch Border Crossing

This list covers the major points and exits along this section of I-5 as you approach the Canadian border.


In [ ]:
#Create a lookup table of mile post and description in order to connect accident location to milepost

import folium

# Create a base map centered on Bellingham, WA area
map_i5 = folium.Map(location=[48.75, -122.45], zoom_start=10)

# Add markers for each milepost location
milepost_locations = {
    241: [48.635, -122.332],  # Near Lake Samish
    242: [48.642, -122.352],  # Nulle Road exit
    243: [48.653, -122.376],  # Chuckanut Drive exit
    250: [48.700, -122.469],  # South Bellingham
    252: [48.721, -122.485],  # Fairhaven Parkway
    253: [48.734, -122.484],  # Lakeway Drive
    254: [48.745, -122.472],  # Iowa Street
    255: [48.758, -122.465],  # Sunset Drive
    256: [48.770, -122.470],  # Meridian Street
    260: [48.828, -122.514],  # Slater Road
    262: [48.856, -122.585],  # Ferndale Main Street
    266: [48.909, -122.632],  # Grandview Road
    270: [48.967, -122.677],  # Birch Bay-Lynden Road
    274: [49.001, -122.744],  # D Street, Blaine
    276: [49.003, -122.747]   # Peace Arch Border Crossing
}

# Add markers to the map
for milepost, coords in milepost_locations.items():
    folium.Marker(
        location=coords,
        popup=f"Milepost {milepost}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(map_i5)

# Save and display the map
map_i5.save('i5_mileposts_241_to_276_map.html')
map_i5


In [130]:
#Create a histogram of mile post

In [131]:
#filter out data less than mile post 241 and greater than mile post 276
data = data[(data['Mile Post'] >= 241) & (data['Mile Post'] <= 276)]

In [132]:
milepost_names = {
    241: 'Near Lake Samish',
    242: 'Nulle Road Exit',
    243: 'Chuckanut Drive exit',
    250: 'South Bellingham',
    252: 'Fairhaven Parkway',
    253: 'Lakeway Drive',
    254: 'Iowa Street',
    255: 'Sunset Drive',
    256: 'Meridian Street',
    260: 'Slater Road',
    262: 'Ferndale Main Street', 
    266: 'Grandview Road', 
    270: 'Birch Bay-Lynden Road', 
    274: 'D Street, Blaine',
    276: 'Peace Arch Border Crossing'
}

# Add a new column to the data with the milepost name according to the closest mile post number in the milepost_names dictionary
data['Mile Post Name'] = data['Mile Post'].apply(lambda x: min(milepost_names.keys(), key=lambda y: abs(x-y)))
data['Mile Post Name'] = data['Mile Post Name'].map(milepost_names)


# Clean Data
1. There are multiple rows which contain "IOWA"
2. There are multiple rows which contain the word I-5 for Primary Roadway
2. Verify that all records that have Primary Trafficway = 'I-5'
3. Look into using milepost instead of Primary and Secondary Trafficway

In [ ]:
#Show all rows where Primary Trafficway contains '5'
data[data['Primary Trafficway'].str.contains('5')]

#Show a unique list of Primary Trafficway where the Primary Trafficway contains '5'
data[data['Primary Trafficway'].str.contains('5')]['Primary Trafficway'].unique()

#Various records containing I-5 ('I5', 'I-5', 'I 5', 'INTERSTATE 5', 'INTERSTATE 5 NB', 'INTERSTATE 5 SB')

In [ ]:
#Plot a histogram of mile post name to see where accidents are most likely to occur
import matplotlib.pyplot as plt

data['Mile Post Name'].value_counts().plot(kind='bar')
plt.xlabel('Mile Post Name')
plt.ylabel('Number of Accidents')
plt.title('Number of Accidents by Mile Post Name')
plt.show()


In [ ]:
#Plot a histogram of mile post to see where accidents are most likely to occur
data['Mile Post'].plot(kind='hist', bins=20)
plt.xlabel('Mile Post')
plt.ylabel('Number of Accidents')
plt.title('Number of Accidents by Mile Post')
plt.show()

In [ ]:
#Plot a time series of accidents by date.  Group date by month and year.  Plot one series for each mile post name
data['Collision Date'] = pd.to_datetime(data['Collision Date'])
data['Year'] = data['Collision Date'].dt.year
data['Month'] = data['Collision Date'].dt.month
data.groupby(['Year', 'Month', 'Mile Post Name']).size().unstack().plot()
plt.xlabel('Year, Month')
plt.ylabel('Number of Accidents')
plt.title('Number of Accidents by Year, Month')
plt.show()

In [ ]:
# Let's search for the word 'IOWA' or 'iowa' in the entire dataset.
# We'll first convert all columns to string for this search.

# Convert all columns to string for searching
accident_data_str = data.apply(lambda x: x.astype(str))

# Check for any occurrence of 'IOWA' or 'iowa' in any column
iowa_rows = accident_data_str.apply(lambda row: row.str.contains('IOWA', case=False)).any(axis=1)

# Add a new column to the original dataset indicating whether 'IOWA' or 'iowa' is present in any of the columns
data['Contains Iowa'] = iowa_rows

# Count how many rows contain 'IOWA' or 'iowa'
iowa_count = iowa_rows.sum()

iowa_count
data

In [ ]:
iowa_street_count = data[data['Secondary Trafficway'] == 'IOWA ST'].shape[0]
iowa_street_count

In [ ]:
# Let's filter the data to the relevant columns for this task: Primary Trafficway, Secondary Trafficway, and Mile Post.
trafficway_data = data[['Primary Trafficway', 'Secondary Trafficway', 'Mile Post']].drop_duplicates()

# Group by Primary and Secondary Trafficway and check for any Mile Post values within one mile.
# To identify duplicates that need to be reconciled, we'll need to sort and then group by trafficways.

# Sort the data by Primary and Secondary Trafficway and Mile Post
trafficway_data = trafficway_data.sort_values(by=['Primary Trafficway', 'Secondary Trafficway', 'Mile Post'])

# Create a function to reconcile Mile Post values that are within one mile
def reconcile_mile_post(group):
    group['Mile Post Diff'] = group['Mile Post'].diff().fillna(0)
    group['Combine'] = group['Mile Post Diff'].abs() <= 1
    return group

# Apply the function to each group of Primary and Secondary Trafficways
reconciled_trafficways = trafficway_data.groupby(['Primary Trafficway', 'Secondary Trafficway']).apply(reconcile_mile_post)

# Display the reconciled trafficways with information about combining
reconciled_trafficways[['Primary Trafficway', 'Secondary Trafficway', 'Mile Post', 'Combine']]


In [ ]:
# Now let's combine the records flagged for combination.
# We'll collapse the records where 'Combine' is True by keeping the first entry of those groups.

# Mark groups that need combining
reconciled_trafficways['Group'] = (~reconciled_trafficways['Combine']).cumsum()

# Combine the groups by taking the first value of Mile Post for each group
combined_trafficways = reconciled_trafficways.groupby('Group').agg({
    'Primary Trafficway': 'first',
    'Secondary Trafficway': 'first',
    'Mile Post': 'min'
}).reset_index(drop=True)

combined_trafficways


In [ ]:
#group data by Primary Trafficway, Secondary Trafficway and count of crashes
grouped = data.groupby(['City', 'Primary Trafficway', 'Secondary Trafficway'])
grouped = grouped.size()
grouped = grouped.reset_index()
grouped.columns = ['City', 'Primary Trafficway', 'Secondary Trafficway', 'Crash Count']
grouped = grouped.sort_values(by='Crash Count', ascending=False)

grouped.head()